In [10]:
import glob
import pandas as pd
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

regras_ignorados = {}
regras_ignorados['TIPOBITO'] = ['NA']
regras_ignorados['SEXO'] = ['I','0']
regras_ignorados['RACACOR'] = ['NA']
regras_ignorados['ESTCIV'] = [9]
regras_ignorados['ESC'] = [9]
regras_ignorados['ESCMAE'] = [9]
regras_ignorados['QTDFILVIVO'] = [99]
regras_ignorados['QTDFILMORT'] = [99]
regras_ignorados['GRAVIDEZ'] = [9]
regras_ignorados['GESTACAO'] = [9]
regras_ignorados['PARTO'] = [8,9]
regras_ignorados['OBITOPARTO'] = [9]
regras_ignorados['OBITOGRAV'] = [8,9]
regras_ignorados['OBITOPUERP'] = [99]
regras_ignorados['ASSISTMED'] = [9]
regras_ignorados['EXAME'] = [9]
regras_ignorados['CIRURGIA'] = [9]
regras_ignorados['NECROPSIA'] = [9]
regras_ignorados['CIRCOBITO'] = [0]
regras_ignorados['ACIDTRAB'] = [9]
regras_ignorados['FONTE'] = [9]
regras_ignorados['TPMORTEOCO'] = [9]
regras_ignorados['FONTEINV'] = [9]
regras_ignorados['ESCMAEAGR1'] = [9]
regras_ignorados['ESCFALAGR1'] = [9]

In [12]:


# incompletude
cont = 0
for f in glob.glob('../data1/dado_1996-2020/*'):
    
    df = pd.read_csv(f)
    ano = df['DTOBITO'] % 10000
    codmun = df['CODMUNOCOR']
        
    df_ignorados = df.copy()
    df_totais = df.isna()

    df_totais = df.isna()
    df_nulos = df_totais.copy()

    df_totais = df_totais.replace(True, 1)
    df_totais = df_totais.replace(False, 1)

    df_totais['ANO'] = ano
    df_totais['CODMUNOCOR'] = codmun

    df_totais = df_totais.groupby(['ANO','CODMUNOCOR']).count().reset_index().melt(id_vars=['ANO','CODMUNOCOR'])
    df_totais.columns = ['ANO','CODMUNOCOR','VARIAVEL','TOTAIS']
        
    df_nulos['CODMUNOCOR'] = codmun
    df_nulos['ANO'] = ano

    df_nulos = df_nulos.groupby(['ANO','CODMUNOCOR']).sum().reset_index().melt(id_vars=['ANO','CODMUNOCOR'])
    df_nulos.columns = ['ANO','CODMUNOCOR','VARIAVEL','NULOS']

    for c in df_ignorados.columns:
        if c in regras_ignorados:
            df_ignorados[c] = df_ignorados[c].isin(regras_ignorados[c])
        else:
            if c not in ['ANO','CODMUNOCOR']:
                df_ignorados.drop(columns=[c], inplace=True)

    df_ignorados['CODMUNOCOR'] = codmun
    df_ignorados['ANO'] = ano

    df_ignorados = df_ignorados.groupby(['ANO','CODMUNOCOR']).sum().reset_index().melt(id_vars=['ANO','CODMUNOCOR'])
    df_ignorados.columns = ['ANO','CODMUNOCOR','VARIAVEL','IGNORADOS']

    df_ignorados = df_ignorados.fillna(0)

    df_totais.set_index(['ANO','CODMUNOCOR','VARIAVEL'], inplace=True)
    df_nulos.set_index(['ANO','CODMUNOCOR','VARIAVEL'], inplace=True)
    df_ignorados.set_index(['ANO','CODMUNOCOR','VARIAVEL'], inplace=True)
        
    x = df_totais.join([df_nulos, df_ignorados], how='left')
    x = x.reset_index()

    x = x[['ANO','CODMUNOCOR','VARIAVEL','NULOS','IGNORADOS','TOTAIS']]

    x = x.fillna(0)  

    x.to_csv('../SIM_dataset/resultados/Incompletude_p{}.csv'.format(cont),
             index=None, compression='gzip')
    
    cont += 1


In [13]:
incompletude = pd.DataFrame()

for f in glob.glob('../SIM_dataset/resultados/Incompletude_p*.csv'):
    df = pd.read_csv(f, compression='gzip')
    incompletude = pd.concat([incompletude, df], axis=0)

incompletude.fillna(0, inplace=True)
incompletude = incompletude[~incompletude.VARIAVEL.isin(['contador','NOVO'])]
incompletude.to_csv('../data1/SIM_Incompletude_1996_2020.csv', index=None)

In [16]:
incompletude['ANO'].unique()

array([2020, 2019, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002,
       2001, 2018, 2000, 1999, 1998, 1997, 1996, 2017, 2016, 2015, 2014,
       2013, 2012, 2011], dtype=int64)

In [18]:
warnings.filterwarnings("ignore")

# aplica as regras para variaveis com opcoes
regras_gerais = {'TIPOBITO':[1,2],
                 'SEXO':[1,2,9,0,'M','F','I'],
                 'RACACOR':[1,2,3,4,5],
                 'ESTCIV':[1,2,3,4,5,9],
                 'ESC':[1,2,3,4,5,9],
                 'LOCOCOR':[1,2,3,4,5,9],
                 'ESCMAE':[1,2,3,4,5,9],
                 'GRAVIDEZ':[1,2,3,9],
                 'GESTACAO':[1,2,3,4,5,6,9],
                 'PARTO':[1,2,9],
                 'OBITOPARTO':[1,2,3,9],
                 'OBITOGRAV':[1,2,9],
                 'OBITOPUERP':[1,2,3,9],
                 'ASSISTMED':[1,2,3,9],
                 'EXAME':[1,2,3,9],
                 'CIRURGIA':[1,2,3,9],
                 'NECROPSIA':[1,2,3,9],
                 'CIRCOBITO':[1,2,3,4,9],
                 'ACIDTRAB':[1,2,9],
                 'FONTE':[1,2,3,4,9],
                 'SERIESCMAE': list(range(1,9)),
                 'TPMORTEOCO':[1,2,3,4,5,8,9],
                 'TPPOS':[1,2],
                 'ATESTANTE': list(range(1,6)),
                 'FONTEINV': [1,2,3,4,6,7,8,9],
                 'ESCMAEAGR1': list(range(1,13)),
                 'ESCFALAGR1': list(range(1,13)),
                }

colunas_implausibilidade = regras_gerais.keys()


# implausibilidade

for f in glob.glob('../data1/dado_1996-2020/*'):
    df = pd.read_csv(f)
    ano = df['DTOBITO'] % 10000
    codmun = df['CODMUNOCOR']
    aux_cols = []
    for c in colunas_implausibilidade:
        if c in df.columns:
            aux_cols.append(c)

    aux = df[aux_cols]

    aux['ANO'] = ano
    aux['CODMUNOCOR'] = codmun

    for col in regras_gerais.keys():
        if col in aux_cols:
            aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & (~aux[col].isin(regras_gerais[col]))
        
    # REGRAS ESPECÍFICAS

    for col in ['IDADE','IDADEMAE']:
        if col not in aux_cols:
            continue
        aux[col] = pd.to_numeric(aux[col], errors='coerce')
        aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 0) | (aux[col] > 120))

    for col in ['QTDFILVIVO','QTDFILMORT']:
        if col not in aux_cols:
            continue
        aux[col] = pd.to_numeric(aux[col], errors='coerce')
        aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & (aux[col] != 99) & ((aux[col] < 0) | (aux[col] > 70))
        
    col = 'PESO'
    if col in aux_cols:
        aux[col] = pd.to_numeric(aux[col], errors='coerce')
        aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 0) | (aux[col] > 11000))

    aux_cols = []

    for c in aux.columns:
        if 'IMPLAUSIVEL' in c:
            aux_cols.append(c)
            
    aux_cols = ['ANO','CODMUNOCOR'] + aux_cols

    df_implausiveis = aux[aux_cols]

    df_implausiveis.fillna(0, inplace=True)

    df_implausiveis = df_implausiveis.groupby(['ANO','CODMUNOCOR']).sum().reset_index().melt(id_vars=['ANO','CODMUNOCOR'])
    df_implausiveis.columns = ['ANO','CODMUNOCOR','VARIAVEL','IMPLAUSIVEIS']    


    df['ANO'] = ano
    df['CODMUNOCOR'] = codmun

    df_totais = df[['ANO','CODMUNOCOR']]
    df_totais['TOTAIS'] = 1

    df_totais = df_totais.groupby(['ANO','CODMUNOCOR'])['TOTAIS'].sum().reset_index()
    df_totais.columns = ['ANO','CODMUNOCOR','TOTAIS']
        
    df_totais.set_index(['ANO','CODMUNOCOR'], inplace=True)
    df_implausiveis.set_index(['ANO','CODMUNOCOR'], inplace=True)
        
    x = df_totais.join([df_implausiveis], how='left')
    x = x.reset_index()

    x = x[['ANO','CODMUNOCOR','VARIAVEL','IMPLAUSIVEIS','TOTAIS']]

    x = x.fillna(0)  
    x.to_csv('../SIM_dataset/resultados/Implausibilidade_p{}.csv'.format(cont),
         index=None, compression='gzip')
    cont += 1

In [20]:
implausibilidade = pd.DataFrame()

for f in glob.glob('../SIM_dataset/resultados/Implausibilidade_p*.csv'):
    df = pd.read_csv(f, compression='gzip')
    implausibilidade = pd.concat([implausibilidade, df], axis=0)

implausibilidade.fillna(0, inplace=True)
implausibilidade = implausibilidade[~implausibilidade.VARIAVEL.isin(['contador','NOVO'])]
implausibilidade.to_csv('../data1/SIM_Implausibilidade_1996-2020.csv', index=None, compression='gzip')

In [22]:
warnings.filterwarnings("ignore")

regras = {}
cont = 0

def convert_date(col):
    
    col = col.fillna(0)
    data_string = col.apply(int).apply(str)
    y = data_string.str[-4:]
    m = data_string.str[-6:-4]
    d = data_string.str[:-6]
    return y.str.cat(m.str.cat(d.str.zfill(2)))

# inconsistencias
for f in glob.glob('../data1/dado_1996-2020/*'):
  
    df = pd.read_csv(f)
    ano = df['DTOBITO'] % 10000
    codmun = df['CODMUNOCOR']

    aux_cols = [
        'DTOBITO',
        'DTNASC',
        'SEXO',
        'OBITOPARTO',
        'OBITOGRAV',
        'OBITOPUERP',
        'LOCOCOR',
        'FONTE'
    ]

    aux = df[aux_cols]
            
    # DTOBITO menor que DTNASC
    regras['DTOBITO_e_DTNASC'] = "Se a data de óbito for menor que a data de nascimento"
    aux['DTOBITO'] = convert_date(aux['DTOBITO'])
    aux['DTNASC'] = convert_date(aux['DTNASC'])
        
    aux['DTOBITO_e_DTNASC_INCONSISTENTES'] = (aux['DTOBITO'] < aux['DTNASC'])
        
    # se SEXO estiver como M ou I e os campos OBITOPARTO, OBITOGRAV, OBITOPUERP estiverem preenchidos
    regras['SEXO_e_OBITO'] = "Se SEXO for diferente de 'M','I' e os campos relativos a óbitos em mulheres estiverem preenchidos"

    obito_preenchido = (~aux['OBITOPARTO'].isna()) | (~aux['OBITOGRAV'].isna()) | (~aux['OBITOPUERP'].isna())
    aux['SEXO_e_OBITO_INCONSISTENTES'] = (aux['SEXO'].isin(['M','I'])) & (obito_preenchido)

    # Se OBITOPARTO preenchido como 3 e OBITOPUERP estiver como 3 ou OBITOGRAV estiver como 1; 
    # Se OBITOPARTO preenchido como 1 ou 2 e OBITOGRAV estiver como 2 ou OBITOPUERP estiver como 1 ou 2; 
    # Se OBITOPARTO preenchido como 9, OBITOGRAV estiver como 1 ou 2 ou OBITOPUERP estiver como 1,2 ou 3
    regras['OBITO_PUERPERIO_GRAVIDEZ'] = "Se OBITOPARTO e OBITOPUERP estiver como 3 ou OBITOGRAV estiver como 1;"
    regras['OBITO_PUERPERIO_GRAVIDEZ'] += "Se OBITOPARTO estiver como 1 ou 2 e OBITOGRAV estiver como 2, ou OBITOPUERP estiver como 1 ou 2;"
    regras['OBITO_PUERPERIO_GRAVIDEZ'] += "Se OBITOPARTO estiver como 9 e OBITOGRAV estiver como 1 ou 2, ou OBITOPUERP estiver como 1, 2 ou 3"

    parte_1 = (aux['OBITOPARTO'] == 3) & ((aux['OBITOPARTO'] == 3) | (aux['OBITOGRAV'] == 1))
    parte_2 = (aux['OBITOPARTO'].isin([1,2])) & ((aux['OBITOGRAV'] == 2) | (aux['OBITOPUERP'].isin([1,2])))
    parte_3 = (aux['OBITOPARTO'] == 9) & ((aux['OBITOGRAV'].isin([1,2])) | (aux['OBITOPUERP'].isin([1,2,3])))
    aux['OBITO_PUERPERIO_GRAVIDEZ_INCONSISTENTES'] = (parte_1) | (parte_2) | parte_3

    # Preechido como 1 e o item Morte durante o puerperio também for preenchido como 1,2 ou 9
    regras['OBITOGRAV_e_OBITOPUERP'] = "Se OBITOGRAV estiver como 1 e OBITOPUERP estiver como 1, 2 ou 9"

    aux['OBITOGRAV_e_OBITOPUERP_INCONSISTENTES'] = (aux['OBITOGRAV'] == 1) & (aux['OBITOPUERP'].isin([1,2,3]))

    # Preenchido como 1 ou 2 e o item morte durante a gravidez estiver preenchido como 1 ou 9
    regras['OBITOPUERP_e_OBITOGRAV'] = "Se OBITOGRAV estiver como 1 ou 2 e OBITOGRAV estiver como 1 ou 9"

    aux['OBITOPUERP_e_OBITOGRAV_INCONSISTENTES'] = (aux['OBITOGRAV'].isin([1,2])) & (aux['OBITOGRAV'].isin([1,9]))

    # Se FONTE diferente de 2 e LOCOCOR for igual a 1
    regras['FONTE_E_LOCOCOR'] = "Se FONTE estiver diferente de 2 e LOCOCOR estiver como 1"

    aux['FONTE_E_LOCOCOR_INCONSISTENTES'] = (aux['FONTE'] != 2) & (aux['LOCOCOR'] == 1)

    aux_cols = []
    for c in aux.columns:
        if 'INCONSISTENTES' in c:
            aux_cols.append(c)
            
    aux = aux[aux_cols]

    aux['ANO'] = ano
    aux['CODMUNOCOR'] = codmun


    df_inconsistentes = aux

    df_inconsistentes.fillna(0, inplace=True)

    df_inconsistentes = df_inconsistentes.groupby(['ANO','CODMUNOCOR']).sum().reset_index().melt(id_vars=['ANO','CODMUNOCOR'])
    df_inconsistentes.columns = ['ANO','CODMUNOCOR','VARIAVEL','INCONSISTENTES']    


    df['ANO'] = ano
    df['CODMUNOCOR'] = codmun

    df_totais = df[['ANO','CODMUNOCOR']]
    df_totais['TOTAIS'] = 1

    df_totais = df_totais.groupby(['ANO','CODMUNOCOR'])['TOTAIS'].sum().reset_index()
    df_totais.columns = ['ANO','CODMUNOCOR','TOTAIS']
        
    df_totais.set_index(['ANO','CODMUNOCOR'], inplace=True)
    df_inconsistentes.set_index(['ANO','CODMUNOCOR'], inplace=True)
        
    x = df_totais.join([df_inconsistentes], how='left')
    x = x.reset_index()

    x = x[['ANO','CODMUNOCOR','VARIAVEL','INCONSISTENTES','TOTAIS']]

    x = x.fillna(0)
 
    x.to_csv('../SIM_dataset/resultados/Inconsistencia_p{}.csv'.format(cont),
             index=None, compression='gzip')
    
    cont += 1

In [23]:
inconsistencias = pd.DataFrame()

for f in glob.glob('../SIM_dataset/resultados/Inconsistencia_p*.csv'):
    df = pd.read_csv(f, compression='gzip')
    inconsistencias = pd.concat([inconsistencias, df], axis=0)

inconsistencias.fillna(0, inplace=True)
inconsistencias = inconsistencias[~inconsistencias.VARIAVEL.isin(['contador','NOVO'])]
inconsistencias.to_csv('../data1/SIM_Inconsistencia_1996-2020.csv', index=None, compression='gzip')